# Entrenamiento de YOLOv8 en Google Colab (Dataset Pre-descomprimido)

Este notebook está adaptado para entrenar un modelo YOLOv8, asumiendo que el dataset ya está descomprimido en Google Drive. Guarda los resultados en Google Drive y tiene capacidad de reanudar el entrenamiento si la sesión se interrumpe.

### 1. Montar Google Drive

Conecta el notebook a tu Google Drive. Aquí es donde leeremos el dataset y guardaremos los resultados y checkpoints.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### 2. Instalar Librerías

Instalamos `ultralytics` para YOLOv8.

In [ ]:
!pip install ultralytics

### 3. Configuración del Entrenamiento

**ACCIÓN REQUERIDA:** Ajusta las rutas y parámetros en la siguiente celda según tu configuración.

In [ ]:
import os
from ultralytics import YOLO

# --- PARÁMETROS DE CONFIGURACIÓN ---

# Asegurarse de que CUDA_VISIBLE_DEVICES esté configurado para usar la primera GPU
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# Ruta base en tu Google Drive donde tienes el proyecto
# DEBES AJUSTAR ESTA RUTA a donde hayas guardado tu dataset
DRIVE_PROJECT_PATH = '/content/drive/MyDrive/Colab Notebooks/mod03-projects/project-xii-computer-vision' # Asegúrate de que esta ruta sea correcta

# --- Configuración del Dataset (ya descomprimido) ---
# Ruta a la carpeta del dataset ya descomprimido en Drive
DATASET_BASE_PATH = os.path.join(DRIVE_PROJECT_PATH, 'dataset_v2_yolov8_obb')

# Ruta al archivo YAML del dataset
DATASET_YAML_PATH = os.path.join(DATASET_BASE_PATH, 'data.yaml')

# Modelo, epochs y nombre del run
MODEL_NAME = os.path.join(DRIVE_PROJECT_PATH, 'models', 'yolov8m.pt') # Ruta explícita al modelo base en Drive
EPOCHS = 80
RUN_NAME = 'yolov8m_obb_80_epochs_colab_run_3' # Nombre del run actualizado

# Parámetros de rendimiento para Colab
BATCH_SIZE = 8
IMG_SIZE = 640
WORKERS = 4
patience = 20 # Early stopping: detiene el entrenamiento si no hay mejora en 20 épocas
cache = True # Cachea las imágenes en RAM para acelerar la carga (si la RAM lo permite)
optimizer = 'AdamW' # Optimizador AdamW para una convergencia más estable
cos_lr = True # Usa decaimiento de learning rate con coseno annealing

# --- FIN DE LA CONFIGURACIÓN ---

# Asegurarse de que la carpeta de resultados exista en Drive
os.makedirs(os.path.join(DRIVE_PROJECT_PATH, 'training_results'), exist_ok=True)

# --- Verificación del Dataset y Modelo Base ---
if not os.path.exists(DATASET_YAML_PATH):
    print(f"ERROR: El archivo data.yaml no se encontró en {DATASET_YAML_PATH}. Asegúrate de que el dataset esté descomprimido en la ruta correcta.")
    raise FileNotFoundError(f"Dataset YAML not found at {DATASET_YAML_PATH}") # Detener la ejecución si el dataset no se encuentra

if not os.path.exists(MODEL_NAME):
    print(f"ADVERTENCIA: El modelo base {MODEL_NAME} no se encontró en la ruta especificada. Ultralytics lo descargará automáticamente.")
    MODEL_NAME = 'yolov8m.pt' # Volver al nombre corto para que Ultralytics lo descargue

print(f"Dataset listo en: {DATASET_BASE_PATH}")

### 4. Entrenar el Modelo

Esta celda inicia o reanuda el entrenamiento. Gracias a `resume=True`, si el script se detiene, puedes volver a ejecutar esta celda y continuará donde lo dejó.

In [ ]:
from ultralytics import YOLO

# Cargar el modelo base (YOLO descargará 'yolov8m.pt' si no existe)
model = YOLO(MODEL_NAME)

# Iniciar el entrenamiento
results = model.train(
    # --- Rutas y Datos ---
    data=DATASET_YAML_PATH,
    project=os.path.join(DRIVE_PROJECT_PATH, 'training_results'), # Guarda los resultados en Drive
    name=RUN_NAME,

    # --- Parámetros de Entrenamiento ---
    epochs=EPOCHS,
    # ONLY IF you want to resume from last checkpoint
    # resume=True,  # <-- ¡LA CLAVE! Reanuda el entrenamiento si encuentra un checkpoint

    # --- Parámetros de Rendimiento y Optimización ---
    batch=BATCH_SIZE,
    imgsz=IMG_SIZE,
    workers=WORKERS,
    patience=patience,
    cache=cache,
    optimizer=optimizer,
    cos_lr=cos_lr,

    # --- Opcionales ---
    save_json=True,
    save_hybrid=True,
    plots=True,
    device='0' # Forzar el uso de la GPU 0
)

print('--- ENTRENAMIENTO FINALIZADO ---')
print(f'Resultados guardados en: {results.save_dir}')

### 5. Evaluar el Modelo Final

Una vez finalizado el entrenamiento completo, puedes ejecutar esta celda para ver las métricas de validación del mejor modelo.

In [ ]:
# Cargar el mejor modelo guardado durante el entrenamiento
best_model_path = os.path.join(DRIVE_PROJECT_PATH, 'training_results', RUN_NAME, 'weights', 'best.pt')
model = YOLO(MODEL_NAME)

# Evaluar en el conjunto de validación
metrics = model.val()

print('--- MÉTRICAS DE VALIDACIÓN ---')
print(f'mAP50-95: {metrics.box.map:.4f}')
print(f'   mAP50: {metrics.box.map50:.4f}')
print(f'   mAP75: {metrics.box.map75:.4f}')